# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore
from azureml.pipeline.steps import AutoMLStep
from azureml.core.model import Model, InferenceConfig
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core import Environment
from azureml.core.webservice import AciWebservice
import requests
import json

print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
pip show azureml-widgets

Name: azureml-widgets
Version: 1.51.0
Summary: Provides fully supported, with interactivity, async auto-updates, and non-blocking cell execution.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: azure-storage-blob, ipywidgets, azureml-telemetry, jinja2, azureml-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
myenv = Environment(name = "myenv")

In [3]:
ws = Workspace.from_config()
myenv.register(workspace=ws)

# choose a name for experiment
experiment_name = 'diabetes-automl'

experiment=Experiment(ws, experiment_name)

In [5]:
dataset = Dataset.get_by_name(ws, name='diabetes')
df = dataset.to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
cluster_name = "mimi-compute"

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

In [6]:
# TODO: Put your automl settings here
automl_settings = { "experiment_timeout_minutes": 30,
                    "max_concurrent_iterations": 5,
                    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             task = "classification",
                             training_data=df,
                             label_column_name="Outcome",  
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [4]:
# TODO: Submit your experiment
from azureml.train.automl.run import AutoMLRun
remote_run = AutoMLRun(experiment = experiment, run_id = 'AutoML_a0374017-59bc-4daa-8829-8ef9c172030b')

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
print(remote_run.get_file_names())

In [ ]:
print(azureml.core.VERSION)

In [5]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
run_details = remote_run.get_details()
print(run_details)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
auto_job, best_auto_model = remote_run.get_output()
print(auto_job)
print(best_auto_model)

In [ ]:
import joblib

# Save the best model
joblib.dump(value=best_auto_model, filename='auto_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = auto_job.register_model(model_name='best-automl-model',
                                      model_path='outputs/model.pkl',
                                      description='Best model from AutoML for diabetes')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
auto_job.download_files(output_directory='automl_output')

env = Environment.from_conda_specification('automl-env', './automl_output/outputs/conda_env_v_1_0_0.yml')

inference = InferenceConfig(environment=env, entry_script='./automl_output/outputs/scoring_file_v_1_0_0.py')

deployment = AciWebservice.deploy_configuration(cpu_cores=1,
                                                       memory_gb=1,
                                                       enable_app_insights=True)

deployment_name = 'auto-diabetes'
service = Model.deploy(workspace=ws,
                       name=deployment_name,
                       models=[model],
                       inference_config=inference,
                       deployment_config=deployment,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
rest_endpoint = service.scoring_uri
headers = {'Content-type': 'application/json'}
response = requests.post(rest_endpoint,
                         headers=headers,
                         json={
  "data": [
    {
      "Pregnancies": 4,
      "Glucose": 121,
      "BloodPressure": 70,
      "SkinThickness": 21,
      "Insulin": 80,
      "BMI": 32.0,
      "DiabetesPedigreeFunction": 0.47,
      "Age": 33
    }
  ]
}
                        )
response.content


In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data =  {
  "data": [
    {
      "Pregnancies": 4,
      "Glucose": 121,
      "BloodPressure": 70,
      "SkinThickness": 21,
      "Insulin": 80,
      "BMI": 32.0,
      "DiabetesPedigreeFunction": 0.47,
      "Age": 33
    }
  ],
  "method": "predict"
}

body = str.encode(json.dumps(data))

url = 'http://01b8aaff-c40d-4b46-adb9-c1d426f1367b.westus2.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

In [ ]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

In [ ]:
# Delete the Webservice and Compute Cluster
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
